In [10]:
!rm -rf ./*
!git clone https://github.com/gasparyanartur/radio-optimizer.git
!mv radio-optimizer/* ./
!rm -rf radio-optimizer

Cloning into 'radio-optimizer'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), 1.18 KiB | 1.18 MiB/s, done.
